In [1]:
import pandas as pd
import numpy as np
import re
import time
import json

import requests as rq
import bs4 as bs4
import tqdm #tqdm é usando para mostrar barra de carregamento
import glob

In [2]:
df = pd.read_json("parsed_videos.json", lines=True)
df.head()

,link,title,query
0,/watch?v=UV5yJtaP9i4,Machine Learning - Introduction | Tamil | 2020,machine+learning
1,/watch?v=SylE9hC5yqg,MySportsAI6 A GUI based approach to Machine Le...,machine+learning
2,/watch?v=I0VJPfe5djo,Machine Learning & its Types,machine+learning
3,/watch?v=OHElkaIkItk,AI Festival Nigeria VC - Carrying out the busi...,machine+learning
4,/watch?v=a6Aeb0_F12U,Machine Learning using Python - Project assign...,machine+learning


In [3]:
#Removendo duplicados
lista_de_links = df['link'].unique()
len(lista_de_links)

1177

In [4]:
url = "https://youtube.com{link}"

for link in lista_de_links:
    urll = url.format(link=link)
    print(urll)
    response = rq.get(urll)
    
    link_name = re.search("v=(.*)", link).group(1)
    
    with open("./dados_brutos/video_{}.html".format(link_name), 'w+', encoding="utf-8") as output:
        output.write(response.text)

#Expressão regular v={.*} está trazendo todos os dados após o "v=". 
#O campo "group(1)" faz que com que apenas o primeiro grupo após o valor "v=" seja trazido. 

https://youtube.com/watch?v=UV5yJtaP9i4
https://youtube.com/watch?v=SylE9hC5yqg
https://youtube.com/watch?v=I0VJPfe5djo
https://youtube.com/watch?v=OHElkaIkItk
https://youtube.com/watch?v=a6Aeb0_F12U
https://youtube.com/watch?v=4oNeAoM8Jpc


From cffi callback <function _verify_callback at 0x000001A5E4CC3048>:
Traceback (most recent call last):
  File "c:\users\joseilton mota\appdata\local\programs\python\python37\lib\site-packages\OpenSSL\SSL.py", line 311, in wrapper
    @wraps(callback)
KeyboardInterrupt


SSLError: HTTPSConnectionPool(host='www.youtube.com', port=443): Max retries exceeded with url: /watch?v=4oNeAoM8Jpc (Caused by SSLError(SSLError("bad handshake: Error([('SSL routines', 'tls_process_server_certificate', 'certificate verify failed')])")))

# Processamento de dados

In [4]:
with open("parsed_video_info.json", 'w+') as output:
    for video_file in tqdm.tqdm_notebook(sorted(glob.glob("./dados_brutos/video*"))):
        with open(video_file, 'r+', encoding="utf-8") as inp:
            page_html = inp.read()
            parsed = bs4.BeautifulSoup(page_html, 'html.parser')
            
            class_watch = parsed.find_all(attrs={"class":re.compile(r"watch")})
            id_watch = parsed.find_all(attrs={"id":re.compile(r"watch")})
            channel = parsed.find_all("a", attrs={"href":re.compile(r"Channel")})
            meta = parsed.find_all("meta")
            
            data = dict()
            
            #Para cada classe que tenha o valor "watch", junta os elementos e cria uma coluna com o nome
            for e in class_watch:
                colname = "_".join(e['class'])
                if "clearfix" in colname:
                    continue
                data[colname] = e.text.strip() #Remove espaços em branco
            
            for e in id_watch:
                colname = e['id']
                data[colname] = e.text.strip()
                
            for e in meta:
                colname = e.get('property')
                if colname is not None:
                    data[colname] = e['content']
            
            for link_num, e in enumerate(channel):
                data["channel_link_{}".format(link_num)] = e['href']
            
            output.write("{}\n".format(json.dumps(data)))
            
#tqdm é usando para mostrar barra de carregamento
#Glob é uma função que lista os arquivos dentro da pasta, semelhante a "os.listdir"
            

c:\users\joseilton mota\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [16]:
### Exemplo do passo acima
e = class_watch[1]
e
#"_".join(e['class'])

#e = id_watch[0]
#e
#e.text.strip()

<div class="content-alignment watch-small" id="player" role="complementary"><div class="player-height" id="theater-background"></div> <div id="player-mole-container">
<div class="hid player-width player-height player-unavailable" id="player-unavailable">
<img alt="" class="icon meh" data-icon="/yts/img/meh7-vflGevej7.png" src="/yts/img/pixel-vfl3z5WfW.gif"/>
<div class="content">
<h1 class="message" id="unavailable-message">
              Este vídeo não está disponível.

    </h1>
<div class="submessage" id="unavailable-submessage">
</div>
</div>
</div>
<div class="player-width player-height off-screen-target player-api" id="player-api" tabindex="-1"></div>
<script>if (window.ytcsi) {window.ytcsi.tick("cfg", null, '');}</script>
<script>var ytplayer = ytplayer || {};ytplayer.config = {"assets":{"css":"\/s\/player\/f676c671\/www-player.css","js":"\/s\/player\/f676c671\/player_ias.vflset\/pt_BR\/base.js"},"attrs":{"id":"movie_player"},"args":{"innertube_context_client_version":"1.2020041

# Verificação

In [8]:
df = pd.read_json("parsed_video_info.json", lines="true")
df.shape

(1177, 123)

In [9]:
pd.set_option("display.max_columns", 500)
df.head(1)

,content-alignment_watch-small,watch-playlist_player-height,watch-queue-header,watch-queue-info,watch-queue-info-icon,watch-queue-title,watch-queue-control-bar_control-bar-button,watch-queue-mole-info,watch-queue-control-bar-icon,watch-queue-icon_yt-sprite,watch-queue-title-container,watch-queue-count,watch-queue-menu_yt-uix-button-menu_yt-uix-button-menu-dark-overflow-action-menu_hid,watch-queue-menu-choice_overflow-menu-choice_yt-uix-button-menu-item,watch-queue-controls,yt-uix-button_yt-uix-button-size-default_yt-uix-button-empty_yt-uix-button-has-icon_control-bar-button_prev-watch-queue-button_yt-uix-button-opacity_yt-uix-tooltip_yt-uix-tooltip,yt-uix-button-icon_yt-uix-button-icon-watch-queue-prev_yt-sprite,yt-uix-button_yt-uix-button-size-default_yt-uix-button-empty_yt-uix-button-has-icon_control-bar-button_play-watch-queue-button_yt-uix-button-opacity_yt-uix-tooltip_yt-uix-tooltip,yt-uix-button-icon_yt-uix-button-icon-watch-queue-play_yt-sprite,yt-uix-button_yt-uix-button-size-default_yt-uix-button-empty_yt-uix-button-has-icon_control-bar-button_pause-watch-queue-button_yt-uix-button-opacity_yt-uix-tooltip_hid_yt-uix-tooltip,yt-uix-button-icon_yt-uix-button-icon-watch-queue-pause_yt-sprite,yt-uix-button_yt-uix-button-size-default_yt-uix-button-empty_yt-uix-button-has-icon_control-bar-button_next-watch-queue-button_yt-uix-button-opacity_yt-uix-tooltip_yt-uix-tooltip,yt-uix-button-icon_yt-uix-button-icon-watch-queue-next_yt-sprite,watch-queue-items-container_yt-scrollbar-dark_yt-scrollbar,watch-queue-items-list,content-alignment_watch-player-playlist,watch-main-col,watch-title-container,watch-title,watch-secondary-actions_yt-uix-button-group,watch-view-count,watch-action-panels_yt-uix-button-panel_hid_yt-card_yt-card-has-padding,watch-time-text,watch-extras-section,watch-meta-item_yt-uix-expander-body,content_watch-info-tag-list,watch-sidebar,watch-playlist_player-height_hid,watch-sidebar-gutter_yt-card_yt-card-has-padding_yt-uix-expander_yt-uix-expander-collapsed,watch-sidebar-section,watch-sidebar-head,watch-sidebar-body,watch-sidebar-separation-line,watch-queue-mole,watch-queue,watch-queue-title-msg,watch-queue-count-msg,watch-queue-loading-template,watch7-container,watch7-main-container,watch7-main,watch7-preview,watch7-content,watch-header,watch7-headline,watch-headline-title,watch7-user-header,watch7-subscription-container,watch8-action-buttons,watch8-secondary-actions,watch8-sentiment-actions,watch7-views-info,watch-action-panels,watch-actions-share-loading,watch-actions-share-panel,watch-actions-rental-required,watch-description,watch-description-content,watch-description-clip,watch-uploader-info,watch-description-text,watch-description-extras,watch-discussion,watch7-sidebar,watch7-sidebar-contents,watch7-sidebar-offer,watch7-sidebar-ads,watch7-sidebar-modules,watch-related,shared-addto-watch-later-login,og:site_name,og:url,og:title,og:image,og:image:width,og:image:height,og:description,al:ios:app_store_id,al:ios:app_name,al:ios:url,al:android:url,al:android:app_name,al:android:package,al:web:url,og:type,og:video:url,og:video:secure_url,og:video:type,og:video:width,og:video:height,og:video:tag,fb:app_id,yt-pl-watch-queue-overlay,watch-channel-brand-div,watch-channel-brand-div-text,watch-actions-transcript-loading,watch-actions-transcript,watch-transcript-container,watch-transcript-not-found,watch-sidebar-live-chat,watch-meta-item,watch-skeleton,watch-page-skeleton,channel_link_0,watch-sidebar-discussion,content-alignment_watch-small_off-screen-trigger,watch7-player-age-gate-content,og:restrictions:age,watch-title_metadata-updateable-title,watch-view-count_metadata-updateable-viewership,watch-time-text_metadata-updateable-date-text,watch-meta-item_has-image,watch-checkout-offers
0,Este vídeo não está disponível.\n\n \n\n\n\...,Fila de exibiçãoFilaFila de exibiçãoFila \nRem...,Fila de exibiçãoFilaFila de exibiçãoFila \nRem...,Fila de exibiçãoFila,,Fila de exibição,Fila de exibiçãoFila \nRemover todosDesconectar,Fila

In [10]:
colunas_selecionadas = ['watch-title', 'watch-view-count', 'watch-time-text', 'content_watch-info-tag-list', 'watch7-headline',
                    'watch7-user-header', 'watch8-sentiment-actions', "og:image", 'og:image:width', 'og:image:height',
                    "og:description", "og:video:width", 'og:video:height', "og:video:tag", 'channel_link_0']

In [11]:
df[colunas_selecionadas].head()

,watch-title,watch-view-count,watch-time-text,content_watch-info-tag-list,watch7-headline,watch7-user-header,watch8-sentiment-actions,og:image,og:image:width,og:image:height,og:description,og:video:width,og:video:height,og:video:tag,channel_link_0
0,Webinar on Python for Machine Learning Day-4-p...,2 visualizações,Publicado em 18 de abr. de 2020,Pessoas e blogs,Webinar on Python for Machine Learning Day-4-p...,Vamsee Krishna Kiran M\n\n\n\n\n\n\n\n\n\n\n\n...,2 visualizações\n\n\n\n\n\n\n\n0\n\nGostou des...,https://i.ytimg.com/vi/-0AKpTYEA6k/maxresdefau...,1280,720,Topics: Introduction to Machine Learning Machi...,960.0,720.0,workflow,NaN
1,Machine Learning Course A To Z || Beginner to ...,174.937 visualizações,Publicado em 10 de ago. de 2018,Educação,Machine Learning Course A To Z || Beginner to ...,Geek's Lesson\n\n\n\n\n\n\n\n\n\n\n\n\n\nCarre...,174.937 visualizações\n\n\n\n\n\n\n\n5.125\n\n...,https://i.ytimg.com/vi/-58kO_zYUGE/maxresdefau...,1280,720,Welcome to this free online class on machine l...,640.0,360.0,Ai and machine learning course,NaN
2,How to Become A Machine Learning Engineer | Ho...,34.239 visualizações,Publicado em 3 de set. de 2018,Educação,#MachineLearningAlgorithms #Datasciencecourse ...,Simplilearn\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nCarre...,34.239 visualizações\n\n\n\n\n\n\n\n769\n\nGos...,https://i.ytimg.com/vi/-5hEYRt8JE0/maxresdefau...,1280,720,"This video on ""How to become a Machine Learnin...",1280.0,720.0,simplilearn,NaN
3,Python For Data Science Full Course - 9 Hours ...,15.845 visualizações,Publicado em 15 de mar. de 2020,Educação,#edureka #PythonEdureka #pythonfordatasciencef...,edureka!\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nCarregan...,15.845 visualizações\n\n\n\n\n\n\n\n647\n\nGos...,https://i.ytimg.com/vi/-6RqxhNO2yY/maxresdefau...,1280,720,🔥Edureka Python Certification Training: https:...,1280.0,720.0,edureka,NaN
4,Michael I. Jordan: Machine Learning: Dynamical...,4.113 visualizações,Publicado em 2 de mai. de 2019,Licença de atribuição Creative Commons (reutil...,#purdue #michaelijordan #engineering\n\n\n\n ...,Purdue Engineering\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,4.113 visualizações\n\n\n\n\n\n\n\n93\n\nGosto...,https://i.ytimg.com/vi/-8yYFdV5SOc/maxresdefau...,1280,720,2019 Purdue Engineering Distinguished Lecture ...,1280.0,720.0,electrical engineer,NaN


In [14]:
#Este formato carrega as informações mais rápido
df[colunas_selecionadas].to_feather("raw_data.feather")